# LGBM - Data Preparation

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pkl
from transformers import AutoTokenizer
import torch
import sys
from nltk.tokenize import word_tokenize
import re

In [2]:
os.chdir("../../")
sys.path.append("lib/textstat/textstat")
sys.path.append("lib/pyspellchecker")

In [5]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.data_tools.feature_engineering import (
    process_paragraph,
    paragraph_feature_engineering,
    process_sentence,
    sentence_feature_engineering,
    generate_tfidf_features,
    # data_preprocessing,
    # remove_punctuation,
    # process_word,
    # expand_contractions,
)
from lib.data_tools.word_feature_engineering import engineer_word_features
from lib.model.inference import ensemble_inference
from lib.data_tools.data import sliding_window
from lib.textstat import textstat
from lib.pyspellchecker.spellchecker import SpellChecker

In [6]:
seed_everything()

## Data Preparation

### Data Loading

In [7]:
train_df = pd.read_csv(Paths.COMPETITION_TRAIN_CSV_PATH)
train_df.shape

(17307, 3)

### Feature Engineerings

#### Paragraph Level

In [6]:
paragraph_csv_path = os.path.join(Paths.ROOT_FEATURE_PATH, "paragraphs.csv")

In [7]:
if not os.path.exists(paragraph_csv_path):
    paragraph_features = process_paragraph(train_df.copy(deep=True))
    assert paragraph_features.essay_id.unique().shape == train_df.essay_id.unique().shape
    print(paragraph_features.shape)
    paragraph_features.to_csv(paragraph_csv_path, index=False)
else:
    paragraph_features = pd.read_csv(paragraph_csv_path)

In [8]:
paragraph_features = paragraph_feature_engineering(paragraph_features)
assert paragraph_features.essay_id.unique().shape == train_df.essay_id.unique().shape
paragraph_features.shape

(17307, 37)

#### Sentence Level

In [9]:
sentence_csv_path = os.path.join(Paths.ROOT_FEATURE_PATH, "sentences.csv")

In [10]:
if not os.path.exists(sentence_csv_path):
    sentence_features = process_sentence(train_df.copy(deep=True))
    assert sentence_features.essay_id.unique().shape == train_df.essay_id.unique().shape
    print(sentence_features.shape)
    sentence_features.to_csv(sentence_csv_path, index=False)
else:
    sentence_features = pd.read_csv(sentence_csv_path)

In [11]:
sentence_features = sentence_feature_engineering(sentence_features)
assert sentence_features.essay_id.unique().shape == train_df.essay_id.unique().shape
sentence_features.shape

(17307, 32)

### Word Features

In [ ]:
word_csv_path = os.path.join(Paths.ROOT_FEATURE_PATH, "word.csv")

In [8]:
if not os.path.exists(word_csv_path):
    word_features = engineer_word_features(train_df.copy(deep=True))
    assert word_features.essay_id.unique().shape == word_features.essay_id.unique().shape
    print(word_features.shape)
    word_features.to_csv(word_csv_path, index=False)
else:
    word_features = pd.read_csv(word_csv_path)

#### TF-IDF Features

A TF-IDF vectorizer is used to convert the essays into numerical features.

In [12]:
vectorizer, tfidf_features = generate_tfidf_features(train_df.copy(deep=True), None)

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


In [13]:
assert tfidf_features.essay_id.unique().shape == train_df.essay_id.unique().shape

In [14]:
with open("output/LGBM/vectorizer.pkl", "wb") as file:
    pkl.dump(vectorizer, file)

#### DeBERTA Predictions

Predictions made using DeBERTA models

In [15]:
deberta_csv_path = os.path.join(Paths.ROOT_FEATURE_PATH, "deberta.csv")

In [16]:
if not os.path.exists(deberta_csv_path):
    tokenizer = AutoTokenizer.from_pretrained(Paths.TOKENIZER_PATH)
    root_model_path = "output/microsoft/deberta-v3-large"
    model_paths = {
        os.path.join(root_model_path, f"microsoft_deberta-v3-large_fold_{f}_best.pth"): 1 / config.n_folds
        for f in range(config.n_folds)
    }
    model_paths
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    df = sliding_window(train_df.copy(deep=True), tokenizer)
    deberta_features = ensemble_inference(
        df,
        tokenizer,
        model_paths,
        device,
        logits=True,
        overall=True,
        model_wise_reduction=False,
    )
    assert deberta_features.essay_id.unique().shape == train_df.essay_id.unique().shape
    print(deberta_features.shape)
    deberta_features.to_csv(deberta_csv_path, index=False)
else:
    deberta_features = pd.read_csv(deberta_csv_path)

#### Combining Features

In [21]:
feature_df_list = [
    paragraph_features,
    sentence_features,
    tfidf_features,
    word_features,
]

all_features = deberta_features.copy(deep=True)

for feature_df in feature_df_list:
    all_features = pd.merge(all_features, feature_df, on="essay_id")

all_features = pd.merge(all_features, train_df[["essay_id", "score"]], on="essay_id")
assert all_features.essay_id.unique().shape == train_df.essay_id.unique().shape
all_features.shape

(17307, 220)

In [22]:
all_features.to_csv(Paths.FEATURE_ENGG_CSV_PATH, index=False)